In [3]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_id = "olafuraron/emotion-english-distilroberta-base-safetensors"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_id, use_safetensors=True)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, top_k=None)

classifier("I love this!")

[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528688922524452},
  {'label': 'neutral', 'score': 0.0057646059431135654},
  {'label': 'anger', 'score': 0.004419790115207434},
  {'label': 'sadness', 'score': 0.0020923963747918606},
  {'label': 'disgust', 'score': 0.0016119946958497167},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

In [7]:
# from transformers import pipeline
# classifier = pipeline("text-classification",
#                       model="olafuraron/emotion-english-distilroberta-base-safetensors",
#                       top_k=None)
# classifier("I love this!")

In [8]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [9]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548417806625366},
  {'label': 'neutral', 'score': 0.16985167562961578},
  {'label': 'sadness', 'score': 0.11640878766775131},
  {'label': 'surprise', 'score': 0.020700618624687195},
  {'label': 'disgust', 'score': 0.0191007312387228},
  {'label': 'joy', 'score': 0.015161218121647835},
  {'label': 'anger', 'score': 0.003935151267796755}]]

In [10]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296023964881897},
  {'label': 'neutral', 'score': 0.1403861790895462},
  {'label': 'fear', 'score': 0.06816205382347107},
  {'label': 'joy', 'score': 0.047942403703927994},
  {'label': 'anger', 'score': 0.00915635284036398},
  {'label': 'disgust', 'score': 0.00262847519479692},
  {'label': 'sadness', 'score': 0.002122161677107215}],
 [{'label': 'neutral', 'score': 0.44937124848365784},
  {'label': 'disgust', 'score': 0.2735914885997772},
  {'label': 'joy', 'score': 0.10908263921737671},
  {'label': 'sadness', 'score': 0.09362708777189255},
  {'label': 'anger', 'score': 0.04047825187444687},
  {'label': 'surprise', 'score': 0.02697012946009636},
  {'label': 'fear', 'score': 0.006879052612930536}],
 [{'label': 'neutral', 'score': 0.6462165117263794},
  {'label': 'sadness', 'score': 0.24273286759853363},
  {'label': 'disgust', 'score': 0.04342271015048027},
  {'label': 'surprise', 'score': 0.02830052189528942},
  {'label': 'joy', 'score': 0.014211475849

In [11]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [12]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [13]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296023964881897},
 {'label': 'neutral', 'score': 0.1403861790895462},
 {'label': 'fear', 'score': 0.06816205382347107},
 {'label': 'joy', 'score': 0.047942403703927994},
 {'label': 'anger', 'score': 0.00915635284036398},
 {'label': 'disgust', 'score': 0.00262847519479692},
 {'label': 'sadness', 'score': 0.002122161677107215}]

In [14]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [15]:
predictions[3]

[{'label': 'fear', 'score': 0.9281684160232544},
 {'label': 'anger', 'score': 0.03219073265790939},
 {'label': 'neutral', 'score': 0.012808679603040218},
 {'label': 'sadness', 'score': 0.008756867609918118},
 {'label': 'surprise', 'score': 0.008597906678915024},
 {'label': 'disgust', 'score': 0.00843181274831295},
 {'label': 'joy', 'score': 0.0010455820010975003}]

In [16]:
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.00915635284036398},
 {'label': 'disgust', 'score': 0.00262847519479692},
 {'label': 'fear', 'score': 0.06816205382347107},
 {'label': 'joy', 'score': 0.047942403703927994},
 {'label': 'neutral', 'score': 0.1403861790895462},
 {'label': 'sadness', 'score': 0.002122161677107215},
 {'label': 'surprise', 'score': 0.7296023964881897}]

In [17]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [18]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [19]:
emotion_scores

{'anger': [0.0641336739063263,
  0.6126191020011902,
  0.0641336739063263,
  0.3514830470085144,
  0.08141259104013443,
  0.23222504556179047,
  0.5381839871406555,
  0.0641336739063263,
  0.3006701171398163,
  0.0641336739063263],
 'disgust': [0.2735914885997772,
  0.3482842743396759,
  0.10400675237178802,
  0.15072226524353027,
  0.18449543416500092,
  0.7271746397018433,
  0.15585508942604065,
  0.10400675237178802,
  0.27948182821273804,
  0.1779264509677887],
 'fear': [0.9281684160232544,
  0.942527711391449,
  0.9723208546638489,
  0.36070725321769714,
  0.09504326432943344,
  0.05136283114552498,
  0.7474284172058105,
  0.40449607372283936,
  0.9155242443084717,
  0.05136283114552498],
 'joy': [0.9327981472015381,
  0.7044211626052856,
  0.7672368884086609,
  0.25188055634498596,
  0.040564440190792084,
  0.04337581992149353,
  0.8725656270980835,
  0.040564440190792084,
  0.040564440190792084,
  0.040564440190792084],
 'sadness': [0.6462165117263794,
  0.8879395127296448,
  0.

In [20]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    #sentences = books["description"][i].split(".")
    sentences = [s.strip() for s in str(books["description"][i]).split(".") if s.strip()][:5]  # only first 5
    predictions = classifier(sentences, batch_size=32, truncation=True)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [20:12<00:00,  4.29it/s] 


In [21]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [22]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.029641,0.338238,0.983973,0.082013,0.697846,0.956065,0.729602,9780002005883
1,0.594469,0.461990,0.935215,0.704421,0.474187,0.051413,0.212221,9780002261982
2,0.041301,0.024569,0.973285,0.767237,0.042176,0.010860,0.009796,9780006178736
3,0.325304,0.125764,0.436339,0.242209,0.732685,0.043272,0.029084,9780006280897
4,0.091732,0.197434,0.095043,0.041146,0.890048,0.475881,0.074878,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.204143,0.053939,0.660489,0.040316,0.807058,0.857254,0.028640,9788172235222
5193,0.058449,0.127941,0.025688,0.400262,0.891073,0.016014,0.227765,9788173031014
5194,0.011246,0.010868,0.314812,0.942169,0.344738,0.060436,0.056820,9788179921623
5195,0.011089,0.080506,0.409870,0.776053,0.929921,0.029765,0.032275,9788185300535


In [23]:
books = pd.merge(books, emotions_df, on = "isbn13")

In [24]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.029641,0.338238,0.983973,0.082013,0.697846,0.956065,0.729602
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.594469,0.461990,0.935215,0.704421,0.474187,0.051413,0.212221
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.041301,0.024569,0.973285,0.767237,0.042176,0.010860,0.009796
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.325304,0.125764,0.436339,0.242209,0.732685,0.043272,0.029084
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.091732,0.197434,0.095043,0.041146,0.890048,0.475881,0.074878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.204143,0.053939,0.660489,0.040316,0.807058,0.857254,0.028640
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.058449,0.127941,0.025688,0.400262,0.891073,0.016014,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.011246,0.010868,0.314812,0.942169,0.344738,0.060436,0.056820
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.011089,0.080506,0.409870,0.776053,0.929921,0.029765,0.032275


In [25]:
books.to_csv("books_with_emotions.csv", index = False)